# Map Wrapper Development for Scipy Differential Equation

Proof of concept notebook for first Scipy, since it only requires map not the o

Error_function is a pass function.
Our "map" takes care of initiating the whole neuron group, calculating the errors and returing the value.

```
params, fits, error = fit_traces(model = model, input_var = 'v', output_var = 'I',\
                                 input = input_traces, output = output_traces,
                                 dt = 0.1*ms, g = [1*nS, 30*nS], E = [-20*mV,100*mV],
                                 tol = 1e-6)
```

In [1]:
import numpy as np
import multiprocessing as mp

In [2]:
# from scipy.optimize import differential_evolution, rosen
# from scipy.optimize._differentialevolution import DifferentialEvolutionSolver

In [3]:
from nevergrad.optimization import optimizerlib
from concurrent import futures
from nevergrad import instrumentation as inst

In [4]:
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod



#### Define Artificial Data for Func Dev

In [5]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])


In [6]:
params2 = np.array([[ 1.12735759e-08,  8.81556360e-02],
 [ 9.04858783e-09,  8.03343489e-02],
 [ 1.60635907e-08,  1.59710651e-03],
 [ 1.66254878e-09,  2.95479385e-02],
 [ 1.80869973e-08,  2.50218013e-02],])

### Input / Output

In [7]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [8]:
# Create target current traces
output_traces = 10*nS*input_traces

In [9]:
input = input_traces
output = output_traces

### Parameters 

In [10]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

### Calculation for inner parameters

In [11]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [12]:
N

3

In [13]:
Ntraces

1

### Setup The Model for Optimization

In [14]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [15]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.02s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [16]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [17]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [18]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

#### Record error  
additional differential equation calculating the error

In [19]:
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

# store the state of the network
store()

In [20]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Dev Map Optimization 

### Create the Dictonaries of Parameters

In [21]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
             
    return d

In [22]:
params

array([[1.80869973e-08, 2.50218013e-02],
       [1.88373085e-08, 9.89559934e-02],
       [1.88373085e-08, 9.89559934e-02]])

In [23]:
d = parameters_dict(params)
d

{'E': array([1.80869973e-08, 1.88373085e-08, 1.88373085e-08]),
 'g': array([0.0250218 , 0.09895599, 0.09895599])}

In [24]:
neurons.get_states()

{'N': array(3),
 'i': array([0, 1, 2], dtype=int32),
 't': 0. * second,
 'dt': 100. * usecond,
 't_in_timesteps': array(0),
 'g': array([0., 0., 0.]) * siemens,
 'E': array([0., 0., 0.]) * volt,
 'total_error': array([0., 0., 0.]) * amp2}

#### Set Neurons state to restored NeuronGroup

In [25]:
restore()
neurons.set_states(d, units=False)
neurons.get_states()

{'N': array(3),
 'i': array([0, 1, 2], dtype=int32),
 't': 0. * second,
 'dt': 100. * usecond,
 't_in_timesteps': array(0),
 'g': array([25.0218013, 98.9559934, 98.9559934]) * msiemens,
 'E': array([18.0869973, 18.8373085, 18.8373085]) * nvolt,
 'total_error': array([0., 0., 0.]) * amp2}

### Run the Simulation with calculated error

In [26]:
run(duration, namespace = {})

#### Return the error

In [27]:
e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
e = mean(e.reshape((N,Ntraces)),axis=1)
array(e)

array([2.04818928e-19, 3.47473674e-18, 3.47473674e-18])

## All of the above into one hard-coded function

In [ ]:
start_scope()

#### model setup

In [ ]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [ ]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [ ]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [ ]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### Model Fitting

In [47]:
def calc_error(params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    
    return array(e)

In [48]:
calc_error(params2)

[[1.12735759e-08 8.81556360e-02]
 [9.04858783e-09 8.03343489e-02]
 [1.60635907e-08 1.59710651e-03]
 [1.66254878e-09 2.95479385e-02]
 [1.80869973e-08 2.50218013e-02]]


array([9.87696585e-19, 5.28401654e-19, 6.58192637e-22, 2.41325473e-21,
       2.04818928e-19])

## Try The Same with Differential Evolution from Nevergrad

In [49]:
start_scope()

In [50]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [51]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [52]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [53]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### Ask and Tel with calc_error Function

#### Setup instrumentation

In [54]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [60]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=10)

#### Use ask to get the parameters

In [61]:
n_samples = 10
params = []
candidates = []

for _ in range(n_samples):
    
    cand = optim.ask()
    candidates.append(cand)
    params.append(list(cand.args))
    
params

[[0.3967107218048024, 3.486083629274972],
 [-1.624432470278361, 6.909907522043067],
 [3.376760914565943, 4.840959277299572],
 [1.5655850913673504, 4.320353545659024],
 [-3.439642941016359, 6.2016523184372225],
 [-2.0283701127369786, 2.083547145622724],
 [0.631566072262328, 7.622725257999506],
 [-1.0376633893741252, 5.9177952556301445],
 [0.28279202859788516, 3.926122775289299],
 [2.5833895037449928, 3.2007007652956103]]

### Calculate error with NetworkGroup

In [62]:
errors = calc_error(np.array(params))
errors

[[ 0.39671072  3.48608363]
 [-1.62443247  6.90990752]
 [ 3.37676091  4.84095928]
 [ 1.56558509  4.32035355]
 [-3.43964294  6.20165232]
 [-2.02837011  2.08354715]
 [ 0.63156607  7.62272526]
 [-1.03766339  5.91779526]
 [ 0.28279203  3.92612278]
 [ 2.5833895   3.20070077]]


array([  1.91259704, 125.99339947, 267.21663039,  45.75008754,
       455.03159614,  17.86080666,  23.17704785,  37.70794451,
         1.23271329,  68.37068464])

### Use tell to give the errors to Nevergrad

In [63]:
for i, candidate in enumerate(candidates):
    value = errors[i]
    optim.tell(candidate, value)
    
    print(candidate, value)

Candidate(args=(0.3967107218048024, 3.486083629274972), kwargs={}) 1.9125970394651817
Candidate(args=(-1.624432470278361, 6.909907522043067), kwargs={}) 125.99339946843654
Candidate(args=(3.376760914565943, 4.840959277299572), kwargs={}) 267.21663038866023
Candidate(args=(1.5655850913673504, 4.320353545659024), kwargs={}) 45.75008754196945
Candidate(args=(-3.439642941016359, 6.2016523184372225), kwargs={}) 455.03159613833134
Candidate(args=(-2.0283701127369786, 2.083547145622724), kwargs={}) 17.860806662116833
Candidate(args=(0.631566072262328, 7.622725257999506), kwargs={}) 23.17704784592288
Candidate(args=(-1.0376633893741252, 5.9177952556301445), kwargs={}) 37.70794450777031
Candidate(args=(0.28279202859788516, 3.926122775289299), kwargs={}) 1.232713293909272
Candidate(args=(2.5833895037449928, 3.2007007652956103), kwargs={}) 68.37068464269542


In [64]:
ans = optim.provide_recommendation()
list(ans.args)

[0.28279202859788516, 3.926122775289299]